# Analysis Pipeline for Pre and Post Comparison

The following analysis pipeline is created to require very little user-input, to make the analyses are straightforward and quick to run as possible. It also ensures the outputs are consistent across all datasets including the 2023 pilot study. However, there are some parameters that are essential to tweak, such as the root path to where you have stored your data, and the selection of the participants and retreats you interested in analysing.

The **metrics included in this pipeline** are currently restricted to the metrics most important for our research question:

* **Absolute Power Band** (decrease in alpha with eyes open for PTSD, though results generally mixed for this)
* **Peak Alpha Frequency** (Global peak power correlates with PTSD symptoms)
* **Signal Diversity (Lempel Ziv complexity)**
* **Source and sensory level connectivity**
* **Network Based Connectivity** (reduced functional connectivity, lower node degree in PTSD), specifically DMN (PTSD reduced connectivity)

Previously, it additionally contained:

* Frontal and/or Parietal EEG asymmetry (greater right than left parietal asymmetry in alpha band power)
* Peak Alpha Frequency (Global peak power correlates with PTSD symptoms)

These can be found in an older version of this script. We also aim for this pipeline to be extended to include:

* **Oscillations vs. Fractal Power**
* **Microstates in the DMN**


These should all be computed within subject pre and post; and also cross-subjects pre and post for significance testing.

Most methods for computing analyses are contained within 'metrics.ipynb' - it is vital that you execute the cell with %run metrics.ipynb, to import these methods.


In [1]:
import sys
import mne
import pandas as pd
import numpy as np
import os.path
from os import path
import pyprep
from mne import Epochs, pick_types, find_events, pick_types, set_eeg_reference, channels, preprocessing
from mne.io import concatenate_raws, read_raw_bdf, read_raw_eeglab, eeglab
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs, corrmap,  annotate_muscle_zscore
import matplotlib.pyplot as plt
from scipy.integrate import simps
from mne.viz import plot_topomap
from matplotlib import cm, colors, colorbar# f
from matplotlib.colors import LinearSegmentedColormap
from fooof.bands import Bands
from scipy import stats
from mne.epochs import equalize_epoch_counts
from tqdm import tqdm
import time


%matplotlib qt   
#means we can visualise the plots within jupyter notebook/google colab


In [15]:
%run metrics.ipynb

## Visualisation parameters
Here you can define your choice of colourmap. This can be custom or from pre-existing.
For pre-existing, we recommend 'seismic' or 'plasma'. The prettier option is 'plasma', however its harder to infer results

In [ ]:
# Define the colors for your custom colormap
pink = (0.8, 0.2, 0.4)
colors = [(0, pink), (0.5, 'white'), (1, 'blue')]  # Interpolate from pink to white to blue

# Create the custom colormap using LinearSegmentedColormap
custom_cmap = LinearSegmentedColormap.from_list('CustomSeismic', colors)

Select colourmap to use below:

In [3]:
# Create a sample plot to visualize your custom colormap
data = np.random.rand(10, 10)  # Sample data
colourmap = 'plasma' #custom_cmap
plt.imshow(data, cmap=colourmap)
plt.colorbar()

## Select which data to use in the analyses

Important steps here include:
- define your root path
- Select the retreats/months you are interested in (all retreats, or just one)
- Select which participants you want to analyse - e.g. do not use participants with particularly noisy data
- Eyes closed or open data (default is closed, as it tends to be cleaner and is what we are using so far)

In [4]:
task = 'eyes_closed' # will be eyes open or eyes closed
visualise = False # whether to print out plots at each stage
if task == 'eyes_open':
    tasklabel = 'eo'
elif task == 'eyes_closed':
    tasklabel = 'ec'
    
months = ['march','august','september']
root = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\onaya\\preprocessed\\epochs\\'

In [5]:
# subject selection
months_subjects = {
    'march': ['02', '03', '04', '05', '07', '08', '09', '11'],
    'august': ['01', '05', '07', '09', '10', '11'],
    'september': ['03', '04', '05', '06'],
}

## Define statistical parameters
For significance testing, we use cluster permutations to deal with multiple comparisons problem. Below the number of permutations and p-value of significance can be defined.
Relevant libraries include:

* mne.stats.permutation_cluster_test(X)
* mne.stats.permutation_t_test(X)
* mne.stats.permutation_cluster_1samp_test(X)  < --- this one is most appropriate

In [6]:
n_perms = 8500 # number of permutations for statistic tests
p = 0.05 #small group sizes so better to use 0.05 than 0.01

## Define bands and their ranges
We use the same ones as in Timmerman's papers to be consistent with the literature (easier to make comparisons). We are most interested in alpha and beta changes.

In [7]:
bands = Bands({'delta' : [1,4],
               'theta': [4,8],
               'alpha': [8, 13],
               'beta': [13, 30],
              'low gamma': [30,45]})

### Load up all subjects and store in a dictionary
Saves time rather than loading them all up for each method. We drop the EOG channel, as it is not relevant to us after preprocessing. This will take quite some time to run.

n_subjects =  months_subjects = {
    'march': ['02', '03', '04', '05', '07', '08', '09', '11'],
    'august': ['01', '05', '07', '09', '10', '11'],
    'september': ['03', '04', '05', '06'],
}

In [8]:
# merge the participants from different months into one dictionary
merged_subjects = [f'{month[0]}_{number}' for month, numbers in months_subjects.items() for number in numbers]

print(merged_subjects)

['m_02', 'm_03', 'm_04', 'm_05', 'm_07', 'm_08', 'm_09', 'm_11', 'a_01', 'a_05', 'a_07', 'a_09', 'a_10', 'a_11', 's_03', 's_04', 's_05', 's_06']


In [9]:

pre_subjects = [] 
post_subjects = []

for month, subs in tqdm(months_subjects.items(),desc="Processing month"):
    print(f"Month: {month}")
    for sub in tqdm(subs,desc=f"Processing Subjects in {month}"):
        print(f"  - {sub}")
        fname = root + month+'\\'  + 'sub-'+sub+'-'+tasklabel+'-'#+condition+'-epo.fif'
        preraw = mne.read_epochs(fname+ 'pre-epo.fif').drop_channels(['EOG'])
        print(preraw)
        print(preraw.info)
        postraw = mne.read_epochs(fname+ 'post-epo.fif').drop_channels(['EOG'])
        pre_subjects.append(preraw)
        post_subjects.append(postraw)
        time.sleep(0.1)

Processing month:   0%|                                                                          | 0/3 [00:00<?, ?it/s]

Month: march



Processing Subjects in march:   0%|                                                              | 0/8 [00:00<?, ?it/s]

  - 02
Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\onaya\preprocessed\epochs\march\sub-02-ec-pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2999.02 ms
        0 CTF compensation matrices available
Not setting metadata
94 matching events found
No baseline correction applied
0 projection items activated
<EpochsFIF |  94 events (all good), 0 – 2.99902 s, baseline off, ~136.7 MB, data loaded,
 '1': 94>
<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, T7, C3, ...
 chs: 62 EEG
 custom_ref_applied: True
 dig: 64 items (3 Cardinal, 61 EEG)
 file_id: 4 items (dict)
 highpass: 0.5 Hz
 lowpass: 45.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 62
 projs: []
 sfreq: 1024.0 Hz
>
Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\onaya\preprocessed\epochs\march\sub-02-ec-post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2999.02 ms
        0 CTF compen


Processing Subjects in march:  12%|██████▋                                              | 1/8 [03:01<21:08, 181.19s/it]

...


Processing Subjects in september:  50%|████████████████████████▌                        | 2/4 [07:22<07:03, 211.65s/it]

  - 05
Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\onaya\preprocessed\epochs\september\sub-05-ec-pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2999.02 ms
        0 CTF compensation matrices available
Not setting metadata
82 matching events found
No baseline correction applied
0 projection items activated
<EpochsFIF |  82 events (all good), 0 – 2.99902 s, baseline off, ~119.2 MB, data loaded,
 '1': 82>
<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, T7, C3, ...
 chs: 62 EEG
 custom_ref_applied: True
 dig: 64 items (3 Cardinal, 61 EEG)
 file_id: 4 items (dict)
 highpass: 0.5 Hz
 lowpass: 45.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 62
 projs: []
 sfreq: 1024.0 Hz
>
Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\onaya\preprocessed\epochs\september\sub-05-ec-post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2999.02 ms
        0 CT


Processing Subjects in september:  75%|████████████████████████████████████▊            | 3/4 [10:14<03:13, 193.52s/it]

  - 06
Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\onaya\preprocessed\epochs\september\sub-06-ec-pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2999.02 ms
        0 CTF compensation matrices available
Not setting metadata
80 matching events found
No baseline correction applied
0 projection items activated
<EpochsFIF |  80 events (all good), 0 – 2.99902 s, baseline off, ~116.3 MB, data loaded,
 '1': 80>
<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, T7, C3, ...
 chs: 62 EEG
 custom_ref_applied: True
 dig: 64 items (3 Cardinal, 61 EEG)
 file_id: 4 items (dict)
 highpass: 0.5 Hz
 lowpass: 45.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 62
 projs: []
 sfreq: 1024.0 Hz
>
Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\onaya\preprocessed\epochs\september\sub-06-ec-post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2999.02 ms
        0 CT


Processing month: 100%|█████████████████████████████████████████████████████████████████| 3/3 [42:50<00:00, 856.90s/it]


# Time averaged EEG Frequency analysis
Timmerman computed using - Conventional spectral analysis was performed using slepian multitapers with spectral smoothing of +/− 0.5 Hz using the Fieldtrip toolbox.
Resulting spectra from both the IRASA algorithm and conventional analysis were divided in the following frequency bands for statistical analysis: Delta (1–4 Hz), theta (4–8 Hz), alpha (8–13 Hz), beta (13–30 Hz) and low gamma (30–45 Hz).





Generally assumed that the spectral power is the same across the entire epoch. We can average over all epochs. (Fieldtrip tutorial: https://natmeg.se/MEEG_course2018/freqanalysis.html)


## Absolute Band Power (ABS)
Calculate the absolute power for the different channels, decompose these into five frequency bands. Plot the results. Currently takes as input continuous data, but the same compute_psd data will also work for epochs. This blog has a nice rationale for the approach --> https://raphaelvallat.com/bandpower.html. 

## ABS: Group Level Changes
### ABS: Paired Sample T-Test
Computes ABP for pre and post for each subject. Paired sample t-test is calculated across the subjects to test for statistical significance pre and post.

##### Topographical Plots Output
These should show the t-statistic which indicates the difference between the pre and post condition.

##### PSD Line Graph Output

Plotting pre and post on graph, following code from: https://mne.discourse.group/t/how-to-plot-psd-at-the-group-level/4912

In [ ]:
#bands = [("Delta", (1, 4)), ("Theta", (4, 8)), ("Alpha", (8, 13)), ("Beta", (13, 30)), ("Gamma", (30, 45))]
   
fig, axes = plt.subplots(1, len(bands), figsize=(20, 10))
absolute_power_changes = {}

for ind, (label, band_def) in enumerate(tqdm(bands,desc="Processing bands ")):
    t_obs, clusters, clusters_pv, band_data, X = process_abs_band(band_def, pre_subjects, post_subjects, merged_subjects, n_perms)
    absolute_power_changes[label] = band_data
    plot_band_results(label, t_obs, colourmap, axes[ind])
    axes[ind].set_title(label)  # Set the title for the subplot
    
fig.suptitle("Absolute power: Post > Pre", fontsize=24)
plt.tight_layout()  
plt.show()



In [ ]:
# Saving the data
np.save("ABS_metrics.npy", X)

In [ ]:
len(absolute_power_changes['delta']['02']['difference'])


## ABS: Individual level changes
Reveals at an individual level the change in absolute power, for the specified bands. All individuals are shown on one plot, to help with identifying trends and comparisons between different retreats.

In [ ]:
plot_individual_bars(absolute_power_changes, 'alpha', 'beta', 'Absolute Power - Individual level')

### 1/f component - see https://iopscience.iop.org/article/10.1088/1741-2552/ac9aaf/pdf

## Oscillatory Power

to be completed...

## Fractal Power

to be completed...

## Lempel Ziv Complexity (LZc)
LZc captures temporal signal diversity of single channels as well as spatial signal diversity across channels.
LZs used for temporal signal diversity only - LZs quantifies the temporal signal diversity of single channels, with a high score for a uniformly random binary string and a low score for a string of zeros only.

## LZc: Paired Sample T-test



In [ ]:
LZc_results = {band: pd.DataFrame() for band in bands.labels}
for band in bands.labels:
    # Common columns for all DataFrames
    columns = ["subjects", f"{band}_pre_LPz", f"{band}_post_LPz", f"{band}_change"]

    LZc_results[band] = pd.DataFrame(columns=columns)
    LZc_results[band]["subjects"] = merged_subjects
 

In [ ]:
LZc_results = {}
fig, axes = plt.subplots(1, len(bands), figsize=(20, 10))
for ind, (label, band_def) in enumerate(tqdm(bands,desc="Processing bands ")):
    print(label)
    LZc_results[label], t_obs, X, LZc_results = process_lpz_band(label, band_def[0], band_def[1], pre_subjects, post_subjects, merged_subjects, LZc_results, len(merged_subjects), n_perms)
    plot_band_results(label, t_obs, colourmap, axes[ind])
    axes[ind].set_title(label)  # Set the title for the subplot
    
fig.suptitle("Lempel Ziv Complexity: Post > Pre", fontsize=24)
plt.tight_layout()  
plt.show()



In [ ]:
fig, axes = plt.subplots(1, len(bands), figsize=(20, 10))
fig.suptitle("Lempel Ziv Complexity: Post > Pre", fontsize=24)
plt.tight_layout()  
plt.show()

### Save LZc analaysis, so can be reloaded easily for visualisation purposes

In [ ]:
# Saving the data
np.save("lempel_diff_metrics.npy", X)
#np.save("lempel_individual.npy", lempel_individual)

## Global LZc

In [ ]:
LZc_results = {}
LZc_df, t_obs, X, LZc_results = global_LPz(pre_subjects, post_subjects, merged_subjects, len(merged_subjects), n_perms)
im, cm =plot_topomap(t_obs, pos= pre_subjects[0].info, ch_type='eeg', cmap=colourmap, show=True)
fig.suptitle("Broadband Lempel Ziv Complexity: Post > Pre", fontsize=24)  
plt.show()


In [ ]:
plot_individual_bars(LZc_results, 'alpha', 'beta', 'LZc - Individual level')

In [ ]:
for individual_data in lempel_individual.values():
    print(individual_data.keys())

In [ ]:

band = 'theta'
data = lempel_individual[band]

# List of individual names
individuals = list(data.keys())

# Extract the 'difference' values for each individual
difference_values = [data[ind]['difference'] for ind in individuals]

# Create a 2D array where rows are subjects and columns are metrics
diff_matrix = np.array(difference_values)

# Create a heatmap
plt.figure(figsize=(12, 8))
cax = plt.imshow(diff_matrix, cmap='seismic', aspect='auto', origin='lower')
plt.title(band + ' difference heatmap')
plt.xlabel('Channels')
plt.ylabel('Subjects')
plt.colorbar(cax, label='Difference')

# Set the x-tick positions and labels
x_labels = pre_subjects[0].info['ch_names']
plt.xticks(range(len(x_labels)), x_labels, rotation=90) 

# Show the plot
plt.show()


In [ ]:
lempel_individual['delta']['m_02']['difference']

In [ ]:
lempel_individual['delta']['m_03']['difference']

# Connectivity Analyses

## Calculating Connectivity in Source Space

This code is designed to compute and visualize connectivity in source space using EEG data and various neuroimaging techniques. The analysis primarily focuses on functional connectivity, which measures the statistical associations between different brain regions. Here's an overview of the key steps involved:

1. **Data Preprocessing:** EEG data from pre and post conditions is loaded and preprocessed. This includes aligning the data with the subject's anatomy, estimating the forward model, and calculating the inverse operator for source localization.

2. **Label Reordering:** Labels representing different brain regions are reordered based on their spatial location in the left hemisphere. This is done to facilitate the visualization of connectivity patterns.

3. **Connectivity Computation:** The code computes connectivity measures, specifically using the 'wPLI2_debiased' method, which assesses phase-based functional connectivity. The connectivity is calculated within specific frequency bands defined by 'band_def'.

4. **Group-Level Analysis:** Connectivity measures are computed for each subject separately and then aggregated for group-level analysis. This provides an insight into how connectivity patterns change between pre and post conditions.

5. **Connectivity Visualization:** The code generates connectivity plots in the form of circular connectivity graphs, allowing for visual inspection of changes in connectivity patterns. This is done for each frequency band and each label representing a brain region.

6. **Statistical Analysis (Optional):** If uncommented, the code also includes statistical analysis to assess the significance of changes in connectivity between pre and post conditions.

7. **Plotting and Output:** The resulting connectivity plots are saved as images, and optional titles and subtitles are added for clarity.

In summary, this code pipeline takes raw EEG data, processes it, computes source-based connectivity, and provides visual and optional statistical insights into the functional connectivity patterns within the brain. The analysis is performed across different frequency bands and brain regions, providing a comprehensive understanding of how connectivity changes in source space under different experimental conditions.

## Source space connectivity analysis

In [10]:
snr = 1.0  # use lower SNR for single epochs
lambda2 = 1.0 / snr ** 2  # related to the regularization applied to the source estimates. A lower SNR results in a higher lambda2 value.
method = "sLORETA"  # use dSPM method (could also be MNE or sLORETA)
sfreq = 1024  # the sampling frequency
con_methods = ['wpli2_debiased', 'ciplv'] # list of connectivity methods to be used for source connectivity analysis. I
#fmin = 8
#fmax = 13

This downloads essential data files and configurations for the "fsaverage" subject in MNE-Python. It sets up the directory for these files, defines the subject name, transformation data, and source space information, and selects the "standard_1005" montage configuration for electrode positions. These preparations are necessary for subsequent EEG/MEG data analysis using MNE-Python.

In [11]:
# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")
montage = mne.channels.make_standard_montage("standard_1005")

0 files missing from root.txt in C:\Users\dedbl\mne_data\MNE-fsaverage-data


C:\Users\dedbl\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


0 files missing from bem.txt in C:\Users\dedbl\mne_data\MNE-fsaverage-data\fsaverage


In [ ]:
# keep just rois
dmn_roi  = ['isthmuscingulate-rh',  'isthmuscingulate-lh',  'medialorbitofrontal-lh', 'medialorbitofrontal-rh',
        'posteriorcingulate-rh','posteriorcingulate-lh','precuneus-rh','precuneus-lh','rostralanteriorcingulate-rh',
        'rostralanteriorcingulate-lh','lateralorbitofrontal-rh','lateralorbitofrontal-lh','parahippocampal-rh',
       'parahippocampal-lh','caudalanteriorcingulate-lh','caudalanteriorcingulate-rh']

#### Defining regions of interest (in this example - the DMN)
https://www.nature.com/articles/s41598-023-31893-1#Sec2
https://spectral-connectivity.readthedocs.io/en/latest/index.html
These DMN areas are selected from the mentioned papers:
- left, right isthmus cingulate,
- left right medial orbito-frontal, 
- left, right posterior cingulate,
- left  right precuneus, 
- left right rostral anterior cingulate,
 - left right lateral orbitofrontal, 
- left right parahippocampal, right caudal anterior cingulate 

### Calculating connectivity for pre and post separately

### Plotting connectivity

### (1) Sorting out labels and layouts

In [12]:
# Get labels for FreeSurfer 'aparc' cortical parcellation with 34 labels/hemi
labels = mne.read_labels_from_annot('fsaverage', parc='aparc')
labels = [label for label in labels if 'unknown' not in label.name]
label_colors = [label.color for label in labels]


Reading labels from parcellation...
   read 35 labels from C:\Users\dedbl\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\dedbl\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot


In [13]:
# First, we reorder the labels based on their location in the left hemi
label_names = [label.name for label in labels]

# Extract left hemisphere labels
lh_labels = [name for name in label_names if name.endswith('lh')]

# Get the y-location of the label
label_ypos = [np.mean(labels[label_names.index(name)].pos[:, 1]) for name in lh_labels]

# Reorder the labels based on their location
sorted_labels = [label for _, label in sorted(zip(label_ypos, lh_labels))]

# For the right hemi
rh_labels = [label[:-2] + 'rh' for label in sorted_labels]

# Save the plot order and create a circular layout
node_order = sorted_labels[::-1] + rh_labels
node_angles = circular_layout(label_names, node_order, start_pos=90, group_boundaries=[0, len(label_names) / 2])

# Plot the graph using node colors from the FreeSurfer parcellation
# We only show the 300 strongest connections.
# Uncomment and provide necessary imports for plotting if needed
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(figsize=(8, 8), facecolor='black', subplot_kw=dict(polar=True)
# condition = post_con_res['wpli2_debiased'] - pre_con_res['wpli2_debiased']

# plot_connectivity_circle(con_res['wpli2_debiased'], label_names, n_lines=300,
# node_angles=node_angles, node_colors=label_colors, title='All-to-All Connectivity(wPLI)', ax=ax)
# fig.tight_layout()


### (2)Computing Connectivity and Generating the plots

You may want to check pyvista is installed - i.e pip install pyvista

In [16]:
fig, axes = plt.subplots(1, 5, figsize=(20, 10), facecolor='black')
folder = "X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\onaya\\connectivity_source\\"
for ind, (label, band_def) in enumerate(bands): 
    print("band: ", label)
    
    fmin, fmax = band_def
    all_sub_pre = []
    all_sub_post = []
    all_con_pre = []
    all_con_post = []
    ts=[]
    ps =[]
    for sub in range(len(merged_subjects)):
        print("Subject: ", sub)
        pre = pre_subjects[sub]
        post =post_subjects[sub]
        pre_con_res, post_con_res = compute_connectivity(pre,post,band_def, con_methods,montage)
        condition = post_con_res['wpli2_debiased'] - pre_con_res['wpli2_debiased']
        all_sub_pre.append(pre_con_res['wpli2_debiased'])
        all_sub_post.append(post_con_res['wpli2_debiased'])
        
        
    pre = np.array(all_sub_pre)
    post = np.array(all_sub_post)
    diff = np.mean(post,axis=0) - np.mean(pre,axis=0)

        
    np.random.seed(42)
    X = np.ones((len(merged_subjects), 16,16, 2))
    X[:, :,:, 0] = pre[:, :,:]
    X[:, :,:, 1] = post[:, :,:]

    X =  X[:, :,:, 1] - X[:, :,:, 0]  # make paired contrast   post minus pre

    t_obs, clusters, clusters_pv, H0 =  perform_permutation_test(X, n_perms)
    print(label, t_obs, clusters_pb, H0)

   # print("STATS ", t_obs, pv)
   # for chan in range(62):
   #     t,p = stats.wilcoxon(X[:,chan], alternative = 'greater')
   #     ts.append(t)
   #     ps.append(p)
    
   # print(label, t_obs, t_obs.min, t_obs.max)
   # t_obs[np.isnan(t_obs)] = 0
    
    plot_connectivity_circle(con=diff, node_names=label_names, show=True, title ='Connectivity in '+label+' averaged over subjects',colormap = 'gnuplot2',vmin=-0.15, vmax = 0.15, facecolor ='white' , textcolor = 'black')
  #  plt.savefig(folder+label+"_global_averaged_conn.jpg", dpi=300)
    axes[ind].set_title(label, {'fontsize' : 20})  
    
    
fig.suptitle('Connectivity post > pre across subjects', fontsize=16)
plt.tight_layout()  
plt.show()

EEG channel type selected for re-referencing
Reading C:\Users\dedbl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...


C:\Users\dedbl\AppData\Local\Temp\ipykernel_4656\1984441078.py:4: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  data.set_eeg_reference(projection=True) # necerssary for inverse modeling


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`


2023-11-07 11:34:08,007 - qdarkstyle - INFO - QSS file sucessfuly loaded.
2023-11-07 11:34:08,020 - qdarkstyle - INFO - Found version patches to be applied.
2023-11-07 11:34:08,021 - qdarkstyle - INFO - Found application patches to be applied.


Using outer_skin.surf for head surface.
Channel types::	eeg: 62
Projecting sensors to the head surface
Source space          : C:\Users\dedbl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : C:\Users\dedbl\anaconda3\lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : C:\Users\dedbl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\dedbl\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  62 EEG channels from info
Head coordi

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.4s finished


Finished.
Computing rank from data with rank=None
    Using tolerance 2.4e-10 (2.2e-16 eps * 62 dim * 1.7e+04  max singular value)
    Estimated rank (eeg): 61
    EEG: rank 61 computed from 62 data channels with 1 projector
    Created an SSP operator (subspace dimension = 1)
    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 62 -> 61
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 227328
[done]
Computing rank from covariance with rank=None
    Using tolerance 1.8e-13 (2.2e-16 eps * 62 dim * 13  max singular value)
    Estimated rank (eeg): 53
    EEG: rank 53 computed from 62 data channels with 1 projector
2 projection items activated
    EEG regularization : 0.1
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank={'eeg': 53}
    Setting small EEG eigenvalues to zero (without PCA)
Converting forward solution to surface orientation
    No patch info available. The standard source space normals 

C:\Users\dedbl\AppData\Local\Temp\ipykernel_4656\1479082334.py:10: RuntimeWarning: fmin=1.000 Hz corresponds to 3.000 < 5 cycles based on the epoch length 3.000 sec, need at least 5.000 sec epochs or fmin=1.667. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 2
Processing epoch : 3 / 74
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 3
Processing epoch : 4 / 74
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 4
Processing epoch : 5 / 74
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 5
Processing epoch : 6 / 74
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 6
Processing epoch : 7 / 74
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 7
Processing epoch : 8 / 74
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 8
Processing epoch : 9 / 74
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 9
Processing epoch : 10 / 74
Extracting time courses for 68 

NameError: name 'subjects' is not defined